# Datetime calculations

In [ ]:
import numpy as np
from datetime import datetime

from _utils import *
from constants import J2000, dS

## Convert Gregorian Date to Julian Date
#### Sourced from:
- https://aa.quae.nl/en/reken/juliaansedag.html
- http://neoprogrammics.com/sidereal_time_calculator/index.php
- L. E. Doggett, “Calendars,” In: P. K. Seidelmann, Ed., Explanatory Supplement to the Astronomical Almanac, US Naval Observatory, University Science Books Company, Mill Valley, 1992.

#### Abbrevations
- JDN: Julian Day Number (Universal Time, starts at 12:00 UTC)
- JD: JDN + f. Julian Day Number + fraction of the day (Universal Time, starts at 12:00 UTC)
- CJD - CJDN: Chronological Julian Date - Chronological Julian Day Number (Local Time, starts at 00:00 LT)

#### Definition of JD, JDN and CJD, CJDN
The zero point of JD (i.e., JD 0.0) corresponds to 12:00 UTC on 1 January −4712 in the Julian calendar.  
The zero point of CJD corresponds to 00:00 (midnight) local time on 1 January −4712.  
JDN 0 corresponds to the period from 12:00 UTC on 1 January −4712 to 12:00 UTC on 2 January −4712.  
CJDN 0 corresponds to 1 January −4712 (the whole day, in local time).

In [ ]:
def gd2jd(Y, M, D, UT=0):
    '''
    Converts a Gregorian UTC datetime to Julian Date.

    Parameters:
    -----------
    Y : int
        Gregorian year.
    M : int
        Gregorian month.
    D : int
        Gregorian day.
    UT : float, optional
        Time in UTC, default value corresponds to 00:00 UTC.

    Returns:
    --------
    JD : float
    '''
    JDN = 367 * Y - 7 * (Y + (M + 9) // 12) // 4 + 275 * M // 9 + D - 730531.5
    JDN += J2000
    f = UT / 24  # Fraction of the day
    return JDN + f

In [ ]:
gd2jd(2000, 1, 1, UT=0)  # 2000.01.01, 00:00 UTC should be 2451544.5

In [ ]:
def gd2jd_2(Y, M, D, UT=0):
    '''
    ALternative way to convert a Gregorian UTC datetime to Julian Date.

    Parameters:
    -----------
    Y : int
        Gregorian year.
    M : int
        Gregorian month.
    D : int
        Gregorian day.
    UT : float, optional
        Time of the day in hours in UTC, default value corresponds to
        00:00 UTC.

    Returns:
    --------
    JD : float
    '''
    c_0 = (M - 3) // 12
    x_4 = Y + c_0
    x_3 = x_4 // 100
    x_2 = x_4 % 100
    x_1 = M - 12 * c_0 - 3
    JDN = (146097 * x_3) // 4 + (36525 * x_2) // 100 + (153 * x_1 + 2) // 5 + D
    JDN += 1721118.5
    f = UT / 24  # Fraction of the day
    return JDN + f

In [ ]:
gd2jd_2(2000, 1, 1, UT=0)  # 2000.01.01, 00:00 UTC should be 2451544.5

## Convert Julian Date to Gregorian Date

In [ ]:
def jd2gd(JD):
    '''
    Convert Julian Date to Gregorian UTC datetime.

    Algorithm is the same as in `jdcal.jd2gcal()` seen in
    https://github.com/phn/jdcal/blob/master/jdcal.py#L193.

    Parameters
    ----------
    JD : scalar or array_like
        Julian Date or array of Julian Dates.
    '''
    f, JDN = np.modf(JD)

    # Set JD to noon of the current date. Fractional part is the
    # fraction from midnight of the current date.
    if -0.5 < f < 0.5:
        f += 0.5
    elif f >= 0.5:
        JDN += 1
        f -= 0.5
    elif f <= -0.5:
        JDN -= 1
        f += 1.5

    # Calculate years, months and days
    ell = np.int64(JDN + 68569)
    n = np.int64((4 * ell) / 146097)
    ell -= np.int64(((146097 * n) + 3) / 4)
    i = np.int64((4000 * (ell + 1)) / 1461001)
    ell -= np.int64((1461 * i) / 4) - 31
    j = np.int64((80 * ell) / 2447.0)
    D = ell - np.int64((2447 * j) / 80)
    ell = np.int64(j / 11)
    M = j + 2 - (12 * ell)
    Y = 100 * (n - 49) + i + ell

    # Calculate UT time of the day as fraction of hours
    UT = f*24

    return Y, M, D, UT

In [ ]:
jd2gd(2451544.5)  # 2451544.5 should be 2000.01.01, 00:00 UTC

## Calculate Greenwich Mean Sidereal Time (GMST $= S_{0}$) on the given date at 00:00 UTC

#### Method 1.
$$
    S_{0} = 24110.54841 + 8640184.812866\,T_{u} + 0.093104\,{T_{u}}^{2} - 6.2 \times 10^{-6}\,{T_{u}}^3,
$$

where $T_{u}$ is number of Julian centuries since J2000.0. $S_{0}$ in this form is in seconds of time.

#### Sourced from:
- https://astronomy.stackexchange.com/questions/21002/how-to-find-greenwich-mean-sideral-time
- http://www2.arnes.si/~gljsentvid10/sidereal.htm
- https://en.wikipedia.org/wiki/Universal_Time#Versions

In [ ]:
def gmst(Y, M, D):
    '''
    Calculates the Greenwich Mean Sidereal Time (GMST = S0) on a given
    date at 00:00 UTC.

    Parameters:
    -----------
    Y : int
        Gregorian year.
    M : int
        Gregorian month.
    D : int
        Gregorian day.

    Returns:
    --------
    GMST : float
        Greenwhich Mean Sidereal Time in hours.
    '''
    JD = gd2jd(Y=Y, M=M, D=D, UT=0)
    T_u = (JD - J2000) / 36525  # Number of Julian centuries since J2000

    c0, c1, c2, c3 = 24110.54841, 8640184.812866, 0.093104, 6.2e-05
    GMST = (c0 + c1 * T_u + c2 * T_u**2 - c3 * T_u**3) / 3600 % 24  # Hours
    return GMST

In [ ]:
print(gmst(2000, 1, 1))  # 2000.01.01, 00:00 UTC should be 6.6645196...

#### Method 2.
$$
    S_{0} = 280.46061837 + 360.98564736629 \times (\text{JD} - \text{J2000})  + 0.000388\,{T_{u}}^{2} - T_{u}^{3}/387100000,
$$

where $T_{u}$ is number of Julian centuries since J2000.0 and $\text{JD}$ is the Julian Date. $S_{0}$ in this form is in arc degrees.

#### References:
- http://www.cashin.net/sidereal/calculation.html

In [ ]:
def gmst_2(Y, M, D):
    '''
    Alternative way to calculate the Greenwich Mean Sidereal Time 
    (GMST = S0) on a given date at 00:00 UTC.

    Parameters:
    -----------
    Y : int
        Gregorian year.
    M : int
        Gregorian month.
    D : int
        Gregorian day.

    Returns:
    --------
    GMST : float
        Greenwhich Mean Sidereal Time in hours.
    '''
    JD = gd2jd(Y=Y, M=M, D=D, UT=0)
    T_u = (JD - J2000) / 36525  # Number of Julian centuries since J2000

    c0, c1, c2, c3 = 280.46061837, 360.98564736629, 0.000388, 387100000
    GMST = (c0 + c1 * (JD-J2000) + c2 * T_u**2 - T_u**3 / c3) / 15 % 360  # Hours
    return GMST

In [ ]:
print(gmst_2(2000, 1, 1))  # 2000.01.01, 00:00 UTC should be 6.6645196...

### Calculate Local Mean Sidereal Time (LMST $= S$) on the given date and time at a specific location
The Local Mean Sidereal Time could be approximated for any given location on earth with the formula

$$
    S = S_{0} + \lambda^{\ast}
$$

Where $\lambda^{\ast}$ is the longitude of the choosen position in hours of time. This value can be made more accurate by taking into account the difference between the Sidereal and Synodic/Solar day. Hence UTC is Synodic, but LMST is Sidereal, we can take into account this with an additional correction:

$$
    S = S_{0} + \lambda^{\ast} + dS \cdot T_{\text{UTC}}
$$

Here $dS = 1.00273790935(\dots)$ indicates the ration between the Sidereal and Synodic day and $T_{\text{UTC}}$ represents the actual UTC time in hours.

#### References:
- https://www.cfa.harvard.edu/~jzhao/times.html
- https://tycho.usno.navy.mil/sidereal.html

In [ ]:
def lmst(Y, M, D, UT=0, long=0):
    '''
    Calculates Local Mean Sidereal Time (LMST = S) on the given date and
    time at a specific location.

    Parameters:
    -----------
    Y : int
        Gregorian year.
    M : int
        Gregorian month.
    D : int
        Gregorian day.
    UT : float, optional
        Time in UTC hours, default value corresponds to 00:00 UTC.
    long : float, optional
        The longitude in degrees of the location the LMST is calculated
        for.

    Returns:
    --------
    LMST : float
        Local Mean Sidereal Time in hours.
    '''
    # Normalize input parameters to their expected ranges
    long = normalize_asym(x=long, p=180)

    GMST = gmst(Y, M, D)
    LMST = GMST + long/15 + dS * UT
    return LMST

In [ ]:
lmst(2000, 1, 1, 0, 0)  # 2000.01.01, 00:00 UTC, 0°N should be 6.6645196...

In [ ]:
lmst(2000, 1, 1, 0, 179.999) - lmst(2000, 1, 1, 0, 180.001)  # Should be ~24